In [1]:
# Code
import sys
sys.path.append('../')
from generative_formalism import *


In [2]:
def save_latex_table_of_period_sample():
    df_smpl = get_chadwyck_corpus_sampled_by('period')
    path = get_path_for_df(df_smpl, is_table=True)
    return get_period_subcorpus_table(df_smpl, save_latex_to=path, return_display=True)

def save_latex_table_of_rhyme_promptings():
    df = get_genai_rhyme_promptings()
    return get_rhyme_promptings_table(df, return_display=True)

def save_latex_table_of_num_poems_per_model():
    df = get_genai_rhyme_promptings()
    return get_num_poems_per_model_table(df, return_display=True)

In [3]:
# save_latex_table_of_period_sample()

In [4]:
# save_latex_table_of_num_poems_per_model()

In [5]:
# save_latex_table_of_num_poems_per_model()

In [6]:
def save_latex_table_of_num_poems_completed_per_model():
    df = get_genai_rhyme_completions(by_line=False)
    df['model2'] = df.model.apply(rename_model)
    df = df[df.model2 != '']
    df_counts = df.groupby(['model2','period']).size().unstack().T
    return get_num_poems_completed_per_model_table(df_counts, return_display=True)


In [7]:
# save_latex_table_of_num_poems_completed_per_model()


In [8]:
def save_latex_table_of_rhyming_accuracy():
    df_smpl = get_chadwyck_corpus_sampled_by('rhyme')
    df_smpl_rhyme = get_rhyme_for_sample(df_smpl, with_sample=True)
    df_preds = get_rhyming_accuracy_by_rhyme_threshold(df_smpl_rhyme)
    df_preds_tbl = df_preds.reset_index().drop('pred_by',axis=1)[['opt','precision','recall','f1_score',]].set_index('opt').sort_index()
    df_preds_tbl.columns = ['Precision', 'Recall', 'F1 score']
    df_preds_tbl.rename_axis('# Rhymes per 10 lines', inplace=True)
    df_preds_tbl = df_preds_tbl.round(2).applymap(lambda x: f'{x*100:.0f}%')
    return get_rhyme_accuracy_table(df_preds_tbl)

In [9]:
# save_latex_table_of_rhyming_accuracy()

In [10]:
def save_latex_table_of_memorization():
    df_mem = get_all_memorization_data()
    df_mem = df_mem[df_mem.found_source_corpus.str.contains('chadwyck')]
    df_mem['found'] = df_mem.found.map({False:'Not found', True:'Found'})
    df_mem['found_source'] = df_mem.found_source.map({'closed':'Closed model output', 'open':'Open training data'})
    df_mem = df_mem.rename(columns={'period':'Period'})
    df_counts = df_mem.groupby(['found_source', 'found', 'Period']).size().unstack().T
    return get_memorization_table(df_counts, return_display=True)


In [11]:
# df_sonnets = get_sonnet_rhythm_data()
# df_sonnets['source2'] = df_sonnets.source.map({'chadwyck':'Historical', 'shakespeare':'Historical', 'genai':'LLM'})

# def get_cent(period):
#     y=int(period[:2])
#     return f'C{y+1}'

# def get_model2(row):
#     if row.source2 == 'LLM':
#         return rename_model(row.model)
#     elif row.source == 'shakespeare':
#         return 'Shakespeare'
#     else:
#         return get_cent(row.period)

# df_sonnets['model2'] = df_sonnets.apply(get_model2, axis=1)
# odf = df_sonnets.groupby(['source2', 'model2']).size().reset_index()
# odf.columns = ['Source', '', '# Sonnets']
# odf = odf.set_index(['Source', ''])
# print(odf)

In [12]:
# In your notebook:
def save_latex_table_of_sonnets():
    df_sonnets = get_sonnet_rhythm_data()
    df_sonnets['source2'] = df_sonnets.source.map({'chadwyck':'Historical', 'shakespeare':'Historical', 'genai':'LLM'})
    
    def get_cent(period):
        y=int(period[:2])
        return f'C{y+1}'
    
    def get_model2(row):
        if row.source2 == 'LLM':
            return rename_model(row.model)
        elif row.source == 'shakespeare':
            return 'Shakespeare'
        else:
            return get_cent(row.period)
    
    df_sonnets['model2'] = df_sonnets.apply(get_model2, axis=1)
    odf = df_sonnets.groupby(['source2', 'model2']).size().reset_index()
    odf.columns = ['Source', '', '# Sonnets']
    odf = odf.set_index(['Source', ''])
    return get_sonnets_table(odf, return_display=True)

In [13]:
save_latex_table_of_sonnets()

* Loading sonnet rhythm data from `{REPO}/data/data_as_in_paper/sonnet_rhythm_data_by_sonnet_period.csv.gz`

* LaTeX compile failed: Command '['/Library/TeX/texbin/pdflatex', '-interaction=nonstopmode', '-halt-on-error', 'table.tex']' returned non-zero exit status 1.

'/Users/ryan/github/generative-formalism/data/data_as_in_paper/tex/table.sonnets.tex'

In [21]:
def save_latex_table_of_text_vs_instruct_completions():
    df = get_text_vs_instruct_completions()
    df_counts = df.groupby(['model','period']).size().unstack().T
    return get_text_vs_instruct_table(df_counts, return_display=True)

In [22]:
save_latex_table_of_text_vs_instruct_completions()

NameError: name 'get_text_vs_instruct_table' is not defined